In [22]:
import pandas as pd
import numpy as np
import gurobipy as grb
from haversine import haversine, Unit

In [23]:
df_income = pd.read_csv("./Data/income.csv")
df_care = pd.read_csv("./Data/child_care.csv")
df_employment = pd.read_csv("./Data/employment.csv")
df_population = pd.read_csv("./Data/population.csv")
df_locations = pd.read_csv("./Data/potential_loc.csv")

In [24]:
## The data set of population of children

df_children = df_population.loc[:, ['zip_code', '-5', '5-9', '10-14']]

## Calculate population in each group
df_children['0-14'] = df_children['-5'] + df_children['5-9'] + df_children['10-14']
df_children['population_total'] = ((13 / 15) * df_children['0-14']).astype(int)
df_children['population_5-12'] = df_children['population_total'] - df_children['-5']
df_children = df_children.drop(['5-9', '10-14', '0-14'], axis = 1).rename({'-5': 'population_0-5'}, axis = 1)

In [25]:
df_locations

,zip_code,latitude,longitude
0,10001,40.741893,-74.000140
1,10001,40.752007,-74.005436
2,10001,40.750545,-73.997147
3,10001,40.744080,-74.001932
4,10001,40.748690,-73.999341
...,...,...,...
102295,14806,42.155072,-77.788830
102296,14806,42.157253,-77.785814
102297,14806,42.161653,-77.783166
102298,14806,42.158862,-77.789563


In [26]:
## The demographical data of each region
df_demo = pd.merge(left = df_income, right = df_employment, how = "left", on = "zip_code").merge(df_children, on = "zip_code")

## Identify high demand regions
df_demo['high demand'] = ((df_demo['average income'] > 60000) * (df_demo['employment rate'] > 0.6)).astype(int)

In [27]:
df_care

,zip_code,facility_id,program_type,facility_status,facility_name,city,school_district_name,infant_capacity,toddler_capacity,preschool_capacity,school_age_capacity,children_capacity,total_capacity,latitude,longitude
0,10001,837597,SACC,Registration,I Have a Dream Foundation,New York,Manhattan 2,0,0,0,84,0,84,40.748836,-73.999810
1,10001,661697,GFDC,License,Chelsea Little Angels Day Care,New York,Manhattan 2,0,0,0,4,12,16,40.748911,-74.001546
2,10001,837329,SACC,Registration,Bright Horizons at Hudson Yards,New York,Manhattan 2,0,0,0,17,0,17,40.752093,-74.002588
3,10001,350076,FDC,Registration,GRAMMAS HANDS,New York,Manhattan 2,0,0,0,2,6,8,40.748296,-74.001263
4,10001,292419,SACC,Registration,The Hudson Guild @26th Street,New York,Manhattan 2,0,0,0,79,0,79,40.749247,-74.001598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14751,14770,115279,SACC,Registration,Young Men's Christian Association of Olean NY ...,Portville,Portville,0,0,15,55,0,70,42.034147,-78.331101
14752,14772,782407,SACC,Registration,"The Relief Zone, Inc.",Randolph,Randolph,0,0,32,60,0,92,42.164507,-78.966681
14753,14772,888218,GFDC,License,Main Street Munchkins,Randolph,Randolph,0,0,0,4,12,16,42.163749,-78.968019
14754,14805,866946,FDC,Registration,Alpine Children's House,Alpine,Trumansburg,0,0,0,2,6,8,NaN,NaN


In [28]:
## Get current capacity
## Can add latitude and longitude to include coordinates
df_care["current_0-5"] = (df_care['infant_capacity'] 
                              + df_care['toddler_capacity'] 
                              + df_care['preschool_capacity'] 
                              + (5/12) * df_care['children_capacity']
                             ).astype(int)

df_care["current_5-12"] = (df_care['total_capacity'] - df_care['current_0-5']).astype(int)

In [29]:
df_current = df_care.loc[:, ["zip_code", "current_0-5", "current_5-12", "total_capacity", 'latitude', 'longitude']]

## For existing facilities with 0 capacity, we cannot estimate the cost of expanding, thus we drop them
df_current = df_current.where(df_current['total_capacity'] != 0).dropna()

"""
df_temp = df_current.merge(df_current['zip_code'].value_counts().rename("count"), right_index = True, left_on = "zip_code")

counts = df_temp['count'].to_list()
curr_idx = 0
curr_count = counts[0]\
n = len(counts)
new_idx = []

for i in range(n):
    if curr_idx == curr_count:
        curr_count = counts[i]
        curr_idx = 0
    
    new_idx.append(curr_idx)
    curr_idx += 1

new_facility_idx = pd.Series(new_idx, name = "new_id")
df_current = df_current.merge(new_facility_idx, how = "left", left_index = True, right_index = True).fillna(0).astype(int)
"""

## Get the upper bound for x + y, if total_capacity > 500, then set the upper bound to 0
df_current['upper_bound'] = 500 - df_current['total_capacity']
df_current['upper_bound'] = df_current['upper_bound'].where(df_current['upper_bound'] > 0).fillna(0).astype(int)
df_current

,zip_code,current_0-5,current_5-12,total_capacity,latitude,longitude,upper_bound
0,10001.0,0.0,84.0,84.0,40.748836,-73.999810,416
1,10001.0,5.0,11.0,16.0,40.748911,-74.001546,484
2,10001.0,0.0,17.0,17.0,40.752093,-74.002588,483
3,10001.0,2.0,6.0,8.0,40.748296,-74.001263,492
4,10001.0,0.0,79.0,79.0,40.749247,-74.001598,421
...,...,...,...,...,...,...,...
14750,14767.0,5.0,11.0,16.0,42.072116,-79.482498,484
14751,14770.0,15.0,55.0,70.0,42.034147,-78.331101,430
14752,14772.0,32.0,60.0,92.0,42.164507,-78.966681,408
14753,14772.0,5.0,11.0,16.0,42.163749,-78.968019,484


In [30]:
## Create the whole data set, containing demographical data, current capacities and requirements by constraints
df_current_capacity = df_current.groupby("zip_code")[["current_0-5", "current_5-12"]].sum().reset_index()
df_main = df_demo.merge(df_current_capacity, on = "zip_code")

## How many 0-5 slots are needed to meet 2/3 population of 0-5 children
df_main['demand_policy'] = (2 * df_main['population_0-5'] / 3).astype(int)

## How many 0-5 slots should be created to meet 2/3 population of 0-5 children
df_main['gap_policy'] = df_main['demand_policy'] - df_main["current_0-5"]

## How many slots are needed to make sure this region not classified as child care desert
df_main['demand_not_desert'] = df_main.apply(
    lambda x: int(x['population_total'] / 2) if x['high demand'] else int(x['population_total'] / 3), 
    axis = 1
)

## How many slots should be created to eliminate child care deserts
df_main['gap_not_desert'] = df_main['demand_not_desert'] - df_main['demand_policy'] - df_main['current_5-12']

## If the gap is negaives, it means no new slots are needed, thus set to 0
df_main['gap_not_desert'] = df_main['gap_not_desert'].where(df_main['gap_not_desert'] > 0).fillna(0)

df_main = df_main.drop(["average income", "employment rate"], axis = 1)\
                 .set_index("zip_code")\
                 .astype(int)
df_main

,population_0-5,population_total,population_5-12,high demand,current_0-5,current_5-12,demand_policy,gap_policy,demand_not_desert,gap_not_desert
zip_code,,,,,,,,,,
10001,744,2140,1396,0,9,600,496,487,713,0
10002,2142,7267,5125,0,93,4628,1428,1335,2422,0
10003,1440,2970,1530,0,0,1995,960,960,990,0
10004,433,672,239,0,0,263,288,288,224,0
10005,484,794,310,1,0,39,322,322,397,36
...,...,...,...,...,...,...,...,...,...,...
14747,156,362,206,0,4,11,104,100,120,5
14767,101,422,321,0,5,11,67,62,140,62
14770,137,482,345,0,15,55,91,76,160,14


In [31]:
# part 2

In [32]:
## Set of zip codes
zip_codes = df_main.index.to_list()

## Set of constraints. type = dict, with zip codes as keys
gap_policy = df_main['gap_policy'].to_dict()
gap_not_desert = df_main['gap_not_desert'].to_dict()

## Set of constants, type = dict, with zip codes as keys
num_facilities = df_current['zip_code'].value_counts().sort_index().to_dict()           ## Number of existing facilities in each region
capacities = df_current.groupby("zip_code")['total_capacity'].apply(list).to_dict()     ## Capacities of existing facilities, categorized by zip codes
upper_bounds = df_current.groupby("zip_code")['upper_bound'].apply(list).to_dict()      ## Upper bounds of x + y

decision_varsX = {k: [0] * v for k, v in num_facilities.items()}        ## Decision variables: x
decision_varsY = {k: [0] * v for k, v in num_facilities.items()}        ## Decision variables: y

decision_varsS = {z: [] for z in zip_codes}                             ## Decision variables: s
decision_varsM = {z: [] for z in zip_codes}                             ## Decision variables: m
decision_varsL = {z: [] for z in zip_codes}                             ## Decision variables: l

In [33]:
import pickle
import numpy as np
import pandas as pd
from haversine import haversine, Unit
import os

distance_threshold = 0.06  # 0.06 miles

# Set this to True if you want to save as an Excel file as well
save_as_xlsx = True  

# Define paths for pickle and Excel files
pickle_path = 'distance_matrices.pkl'
excel_path = 'distance_matrices.xlsx'

# Initialize dictionaries to hold distance matrices
dist_existing_new = {}
dist_new_new = {}

# Check if the pickle file exists
if os.path.exists(pickle_path):
    # Load data from the pickle file
    with open(pickle_path, 'rb') as f:
        distance_data = pickle.load(f)
        
        # Ensure keys exist in the pickle data
        dist_existing_new = distance_data.get('existing_new', {})
        dist_new_new = distance_data.get('new_new', {})

    if dist_existing_new and dist_new_new:
        print("Loaded distance matrices from pickle file.")
    else:
        print("Error: Missing expected keys in pickle file. Loading from Excel instead if available.")
        # Continue to check Excel or create if necessary

# If no pickle file, check for the Excel file
elif os.path.exists(excel_path):
    # Load data from the Excel file and convert to numpy arrays
    xls = pd.ExcelFile(excel_path)
    for i in zip_codes:
        df_existing_new = pd.read_excel(xls, sheet_name=f'existing_new_{i}')
        df_new_new = pd.read_excel(xls, sheet_name=f'new_new_{i}')
        
        dist_existing_new[i] = df_existing_new.values
        dist_new_new[i] = df_new_new.values

    # Save the loaded data to a pickle file for future use
    with open(pickle_path, 'wb') as f:
        pickle.dump({'existing_new': dist_existing_new, 'new_new': dist_new_new}, f)
    print("Loaded distance matrices from Excel and saved to pickle file.")

# If neither pickle nor Excel file exists, create distance matrices
else:
    for i in zip_codes:
        # Extract coordinates for existing and new facilities
        existing_locations = df_current[df_current['zip_code'] == i][['latitude', 'longitude']].values
        new_locations = df_locations[df_locations['zip_code'] == i][['latitude', 'longitude']].values

        # Create distance matrix for existing vs new facilities
        dist_existing_new[i] = np.zeros((num_facilities[i], 100))
        for j in range(num_facilities[i]):
            loc1 = (existing_locations[j][0], existing_locations[j][1])
            for k in range(100):
                loc2 = (new_locations[k][0], new_locations[k][1])
                dist_existing_new[i][j, k] = haversine(loc1, loc2, unit=Unit.MILES)

        # Create distance matrix for new vs new facilities
        dist_new_new[i] = np.zeros((100, 100))
        for k1 in range(100):
            loc1 = (new_locations[k1][0], new_locations[k1][1])
            for k2 in range(k1 + 1, 100):
                loc2 = (new_locations[k2][0], new_locations[k2][1])
                dist_new_new[i][k1, k2] = haversine(loc1, loc2, unit=Unit.MILES)
                dist_new_new[i][k2, k1] = dist_new_new[i][k1, k2]  # Symmetric

    # Save the distance matrices to pickle
    with open(pickle_path, 'wb') as f:
        pickle.dump({'existing_new': dist_existing_new, 'new_new': dist_new_new}, f)
    print("Created and saved distance matrices to 'distance_matrices.pkl'.")

    # Optionally save distance matrices to Excel
    if save_as_xlsx:
        with pd.ExcelWriter(excel_path) as writer:
            for i in zip_codes:
                df_existing_new = pd.DataFrame(dist_existing_new[i])
                df_new_new = pd.DataFrame(dist_new_new[i])

                df_existing_new.to_excel(writer, sheet_name=f'existing_new_{i}', index=False)
                df_new_new.to_excel(writer, sheet_name=f'new_new_{i}', index=False)
                print(f"Saved matrices for zip code: {i} to Excel.")


Loaded distance matrices from pickle file.


In [34]:
from gurobipy import Model, GRB, quicksum

model_pt2 = Model(name="Part2")

# Set of zip codes
zip_codes = df_main.index.to_list()

# Parameters
gap_policy = df_main['gap_policy'].to_dict()
gap_not_desert = df_main['gap_not_desert'].to_dict()
num_facilities = df_current['zip_code'].value_counts().sort_index().to_dict()
capacities = df_current.groupby("zip_code")['total_capacity'].apply(list).to_dict()
upper_bounds = df_current.groupby("zip_code")['upper_bound'].apply(list).to_dict()

# Decision Variables for Expansion
decision_varsX = {k: [None] * v for k, v in num_facilities.items()}
decision_varsY = {k: [None] * v for k, v in num_facilities.items()}

# Decision Variables for New Facilities
decision_varsS = {z: [] for z in zip_codes}
decision_varsM = {z: [] for z in zip_codes}
decision_varsL = {z: [] for z in zip_codes}

# Define decision variables for existing facilities
for z in zip_codes:
    num_facs = num_facilities[z]
    decision_varsX[z], decision_varsY[z] = [None] * num_facs, [None] * num_facs
    for j in range(num_facs):
        x_temp = model_pt2.addVar(lb=0, vtype=GRB.INTEGER, name=f"X_{z}_{j}")
        y_temp = model_pt2.addVar(lb=0, vtype=GRB.INTEGER, name=f"Y_{z}_{j}")
        decision_varsX[z][j] = x_temp
        decision_varsY[z][j] = y_temp

# Variables for new facilities, with small (S), medium (M), and large (L) options
for z in zip_codes:
    for k in range(100):
        s_temp = model_pt2.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"S_{z}_{k}")
        m_temp = model_pt2.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"M_{z}_{k}")
        l_temp = model_pt2.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"L_{z}_{k}")
        decision_varsS[z].append(s_temp)
        decision_varsM[z].append(m_temp)
        decision_varsL[z].append(l_temp)

# Update model
model_pt2.update()

In [35]:
# Define piecewise variables for each facility
delta1 = {i: [model_pt2.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"delta1_{i}_{j}") for j in range(num_facilities[i])] for i in zip_codes}
delta2 = {i: [model_pt2.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"delta2_{i}_{j}") for j in range(num_facilities[i])] for i in zip_codes}
delta3 = {i: [model_pt2.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"delta3_{i}_{j}") for j in range(num_facilities[i])] for i in zip_codes}

# Define binary variables to indicate if each tier is active
is_tier1_active = {i: [model_pt2.addVar(vtype=GRB.BINARY, name=f"is_tier1_active_{i}_{j}") for j in range(num_facilities[i])] for i in zip_codes}
is_tier2_active = {i: [model_pt2.addVar(vtype=GRB.BINARY, name=f"is_tier2_active_{i}_{j}") for j in range(num_facilities[i])] for i in zip_codes}
is_tier3_active = {i: [model_pt2.addVar(vtype=GRB.BINARY, name=f"is_tier3_active_{i}_{j}") for j in range(num_facilities[i])] for i in zip_codes}

# Constraints for Piecewise Segments
M = 1000000000  # Big M constant
for i in zip_codes:
    for j in range(num_facilities[i]):
        # Upper bounds for each delta segment
        model_pt2.addConstr(delta1[i][j] <= 0.1 * capacities[i][j])
        model_pt2.addConstr(delta2[i][j] <= 0.05 * capacities[i][j])
        model_pt2.addConstr(delta3[i][j] <= 0.05 * capacities[i][j])

        # Total expansion equals the sum of piecewise increments
        model_pt2.addConstr(delta1[i][j] + delta2[i][j] + delta3[i][j] == decision_varsX[i][j] + decision_varsY[i][j])

        # Link delta variables to binary indicators
        model_pt2.addConstr(delta1[i][j] <= M * is_tier1_active[i][j])
        model_pt2.addConstr(delta2[i][j] <= M * is_tier2_active[i][j])
        model_pt2.addConstr(delta3[i][j] <= M * is_tier3_active[i][j])

        # Ensure binary variables are activated only when corresponding delta is positive
        model_pt2.addConstr(is_tier1_active[i][j] + is_tier2_active[i][j] + is_tier3_active[i][j] <= 1)

# Objective Function with Piecewise Cost Model
model_pt2.setObjective(
    quicksum(
        sum(
            # Baseline costs for each tier based on binary activation variables
            (20000 * is_tier1_active[i][j] + 200 * capacities[i][j] * delta1[i][j] +
             400 * capacities[i][j] * delta2[i][j] +
             1000 * capacities[i][j] * delta3[i][j] +
             100 * decision_varsX[i][j])  # Additional cost per slot for ages 0-5
            for j in range(num_facilities[i])
        ) for i in zip_codes
    ) +
    quicksum(
        sum(
            65000 * decision_varsS[i][k] + 95000 * decision_varsM[i][k] + 115000 * decision_varsL[i][k]
            for k in range(100)
        ) for i in zip_codes
    ),
    GRB.MINIMIZE
)

In [36]:
# Capacity constraints based on policy gaps
model_pt2.addConstrs(
    (grb.quicksum(decision_varsX[i]) +
     grb.quicksum(50 * decision_varsS[i][k] + 100 * decision_varsM[i][k] + 200 * decision_varsL[i][k] for k in range(100))
     >= gap_policy[i] for i in zip_codes),
    name="policy"
)

# Constraints for no desert regions
model_pt2.addConstrs(
    (grb.quicksum(decision_varsX[i]) + grb.quicksum(decision_varsY[i]) +
     grb.quicksum(100 * decision_varsS[i][k] + 200 * decision_varsM[i][k] + 400 * decision_varsL[i][k] for k in range(100))
     >= gap_not_desert[i] for i in zip_codes),
    name="not_desert"
)

# Constraints for expansion limits by facility
for z, l in num_facilities.items():
    model_pt2.addConstrs(
        (decision_varsX[z][j] + decision_varsY[z][j] <= upper_bounds[z][j] for j in range(l)),
        name="total_capacity"
    )
    model_pt2.addConstrs(
        (5 * (decision_varsX[z][j] + decision_varsY[z][j]) <= capacities[z][j] for j in range(l)),
        name="ratio"
    )
    
# Constraint: Only one new facility (small, medium, or large) can be built in each location
for i in zip_codes:
    for j in range(100):  # assuming there are up to 100 possible new facility locations per zip code
        model_pt2.addConstr(
            decision_varsS[i][j] + decision_varsM[i][j] + decision_varsL[i][j] <= 1,
            name=f"one_facility_per_location_{i}_{j}"
        )

# Distance limitations for new and existing facilities
for z in zip_codes:
    existing_facs = len(dist_existing_new[z])
    new_facs = len(dist_new_new[z])

# Constraints between existing and new facilities
for i in range(existing_facs):
    for j in range(new_facs):
        if dist_existing_new[z][i, j] < 0.06:
            model_pt2.addConstr(
                 decision_varsS[z][j] + decision_varsM[z][j] + decision_varsL[z][j] == 0,
                 name=f"dist_existing_new_{z}_{i}_{j}"
            )

    # Constraints among new facilities
for i in range(new_facs):
    for j in range(i + 1, new_facs):
        if dist_new_new[z][i, j] < 0.06:
            model_pt2.addConstr(
                decision_varsS[z][i] + decision_varsM[z][i] + decision_varsL[z][i] +
                decision_varsS[z][j] + decision_varsM[z][j] + decision_varsL[z][j] <= 1,
                name=f"dist_new_new_{z}_{i}_{j}"
            )

# Solve the model
model_pt2.optimize()
# Check if the model found an optimal solution
if model_pt2.status == grb.GRB.OPTIMAL:
    for z in zip_codes:
        for j in range(num_facilities[z]):
            # Ensure decision_varsX and decision_varsY are Gurobi variables
            if decision_varsX[z][j].x > 0 or decision_varsY[z][j].x > 0:
                print(f"Expand facility in zip code {z}, facility {j}: X = {decision_varsX[z][j].x}, Y = {decision_varsY[z][j].x}")
        for k in range(100):
            # Print information for new facilities
            if decision_varsS[z][k].x > 0 or decision_varsM[z][k].x > 0 or decision_varsL[z][k].x > 0:
                print(f"New facility in zip code {z}, location {k}: Small = {decision_varsS[z][k].x}, Medium = {decision_varsM[z][k].x}, Large = {decision_varsL[z][k].x}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 245023 rows, 415916 columns and 1247979 nonzeros
Model fingerprint: 0x681b6297
Variable types: 42681 continuous, 373235 integer (344781 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  Objective range  [1e+02, 9e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 8e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 4.350750e+08
Presolve removed 107925 rows and 119945 columns (presolve time = 5s) ...
Presolve removed 107925 rows and 143745 columns (presolve time = 10s) ...
Presolve removed 159691 rows and 196905 columns (presolve time = 15s) ...
Presolve removed 244757 rows and 415361 columns (presolve time = 49s) ...
Presolve removed 244757 rows and 415361

In [37]:
if model_pt2.Status == grb.GRB.OPTIMAL:
    print(f"The minimal funding is {model_pt2.ObjVal:.2f}")

The minimal funding is 339088899.88


In [61]:
# Previous answer : The minimal funding is 320307702.23
# We now have : The minimal funding is 339088899.88

In [74]:
if model_pt2.Status == grb.GRB.OPTIMAL:
    print(f"The minimal funding is ${model_pt2.ObjVal:.2f}")


The minimal funding is $339088899.88


In [75]:
# Retrieving results of expansion
zip_code_cols = []
x_cols = []
y_cols = []

for z in zip_codes:
    for j in range(num_facilities[z]):
        zip_code_cols.append(z)
        x_cols.append(decision_varsX[z][j].x)
        y_cols.append(decision_varsY[z][j].x)



In [76]:
df_expand = pd.DataFrame({
    "zip_code": zip_code_cols,
    "expand_0-5": x_cols,
    "expand_5-12": y_cols
})

In [77]:
df_updated = df_current.drop("upper_bound", axis = 1)\
                       .merge(df_expand.drop("zip_code", axis = 1), left_index = True, right_index = True)

df_expand_grouped = df_updated.groupby("zip_code")[["expand_0-5", "expand_5-12"]].sum().reset_index()
df_expand_grouped

,zip_code,expand_0-5,expand_5-12
0,10001.0,0.0,0.0
1,10002.0,0.0,0.0
2,10003.0,0.0,0.0
3,10004.0,0.0,0.0
4,10005.0,0.0,0.0
...,...,...,...
948,14227.0,0.0,0.0
949,14228.0,0.0,0.0
950,14301.0,0.0,0.0
951,14303.0,0.0,0.0


In [78]:
df_updated

,zip_code,current_0-5,current_5-12,total_capacity,latitude,longitude,expand_0-5,expand_5-12
0,10001.0,0.0,84.0,84.0,40.748836,-73.999810,0.0,0.0
1,10001.0,5.0,11.0,16.0,40.748911,-74.001546,0.0,0.0
2,10001.0,0.0,17.0,17.0,40.752093,-74.002588,0.0,0.0
3,10001.0,2.0,6.0,8.0,40.748296,-74.001263,0.0,0.0
4,10001.0,0.0,79.0,79.0,40.749247,-74.001598,0.0,0.0
...,...,...,...,...,...,...,...,...
14221,14303.0,10.0,0.0,10.0,43.086670,-79.020885,0.0,0.0
14223,14304.0,0.0,100.0,100.0,43.084443,-78.973353,0.0,0.0
14224,14304.0,185.0,17.0,202.0,43.091888,-78.907078,0.0,0.0
14225,14304.0,36.0,16.0,52.0,43.099935,-78.977003,0.0,0.0


In [79]:
(df_updated[["expand_0-5", "expand_5-12"]] != 0).sum()

expand_0-5     25
expand_5-12     5
dtype: int64

In [80]:
df_expand_grouped.sum()

zip_code       1.164176e+07
expand_0-5     2.300001e+01
expand_5-12    1.000000e+01
dtype: float64

In [81]:
df_temp = pd.merge(
    left = df_main.reset_index().drop(["demand_policy", 
                                       "gap_policy", 
                                       "demand_not_desert", 
                                       "gap_not_desert",
                                       "population_5-12"
                                       ], axis = 1),
    right = df_expand_grouped,
    on = "zip_code",
    how = "left"
)

In [83]:
zip_code_cols = []
small_new = []
medium_new = []
large_new = []

for z in zip_codes:
    for k in range(100):
        zip_code_cols.append(z)
        small_new.append(decision_varsS[z][k].x)
        medium_new.append(decision_varsM[z][k].x)
        large_new.append(decision_varsL[z][k].x)

df_new_locs = pd.DataFrame({
    "zip_code": zip_code_cols,
    "small": small_new,
    "medium": medium_new,
    "large": large_new
})

df_new = df_new_locs.groupby("zip_code")[["small", "medium", "large"]].sum().reset_index()

In [84]:
data_temp = df_locations.merge(df_new_locs.drop("zip_code", axis = 1), left_index = True, right_index = True)
df_new_grouped = data_temp.groupby("zip_code").agg(
                                                    latitude = ("latitude", "mean"),
                                                    longitude = ("longitude", "mean"),
                                                    small = ("small", "sum"),
                                                    medium = ("medium", "sum"),
                                                    large = ("large", "sum")
                                                ).reset_index()

df_new_grouped["new_0-5"] = df_new_grouped["small"] * 50 + df_new_grouped["medium"] * 100 + df_new_grouped["large"] * 200
df_new_grouped["new_total"] = df_new_grouped["small"] * 100 + df_new_grouped["medium"] * 200 + df_new_grouped["large"] * 400
df_new_grouped

,zip_code,latitude,longitude,small,medium,large,new_0-5,new_total
0,10001,40.748323,-73.997539,0.0,1.0,2.0,500.0,1000.0
1,10002,40.715579,-73.987763,0.0,0.0,7.0,1400.0,2800.0
2,10003,40.730699,-73.988734,0.0,0.0,5.0,1000.0,2000.0
3,10004,40.715242,-74.005619,0.0,1.0,1.0,300.0,600.0
4,10005,40.705546,-74.007167,0.0,0.0,2.0,400.0,800.0
...,...,...,...,...,...,...,...,...
1002,14586,43.040930,-77.687060,0.0,1.0,0.0,100.0,200.0
1003,14589,43.242386,-77.169969,0.0,1.0,0.0,100.0,200.0
1004,14590,43.233995,-76.821190,0.0,1.0,0.0,100.0,200.0
1005,14608,43.151728,-77.626069,0.0,0.0,1.0,200.0,400.0


In [85]:
df_new_grouped.drop(["zip_code", "latitude", "longitude"], axis = 1).sum()

small        3.230000e+02
medium       2.670000e+02
large        2.541000e+03
new_0-5      5.510500e+05
new_total    1.102100e+06
dtype: float64

In [86]:
df_temp

,zip_code,population_0-5,population_total,high demand,current_0-5,current_5-12,expand_0-5,expand_5-12
0,10001,744,2140,0,9,600,0.0,0.0
1,10002,2142,7267,0,93,4628,0.0,0.0
2,10003,1440,2970,0,0,1995,0.0,0.0
3,10004,433,672,0,0,263,0.0,0.0
4,10005,484,794,1,0,39,0.0,0.0
...,...,...,...,...,...,...,...,...
1002,14747,156,362,0,4,11,NaN,NaN
1003,14767,101,422,0,5,11,NaN,NaN
1004,14770,137,482,0,15,55,NaN,NaN
1005,14772,256,635,0,37,71,NaN,NaN


In [87]:
df_results = pd.merge(
    left = df_temp,
    right = df_new_grouped,
    on = "zip_code"
)

df_results["updated_0-5"] = df_results["current_0-5"] + df_results["expand_0-5"] + df_results["new_0-5"]
df_results["updated_total"] = df_results["current_0-5"] + df_results["current_5-12"] + df_results["expand_0-5"] + df_results["expand_5-12"] + df_results["new_total"]
df_results

,zip_code,population_0-5,population_total,high demand,current_0-5,current_5-12,expand_0-5,expand_5-12,latitude,longitude,small,medium,large,new_0-5,new_total,updated_0-5,updated_total
0,10001,744,2140,0,9,600,0.0,0.0,40.748323,-73.997539,0.0,1.0,2.0,500.0,1000.0,509.0,1609.0
1,10002,2142,7267,0,93,4628,0.0,0.0,40.715579,-73.987763,0.0,0.0,7.0,1400.0,2800.0,1493.0,7521.0
2,10003,1440,2970,0,0,1995,0.0,0.0,40.730699,-73.988734,0.0,0.0,5.0,1000.0,2000.0,1000.0,3995.0
3,10004,433,672,0,0,263,0.0,0.0,40.715242,-74.005619,0.0,1.0,1.0,300.0,600.0,300.0,863.0
4,10005,484,794,1,0,39,0.0,0.0,40.705546,-74.007167,0.0,0.0,2.0,400.0,800.0,400.0,839.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,14586,386,1934,0,11,29,NaN,NaN,43.040930,-77.687060,0.0,1.0,0.0,100.0,200.0,NaN,NaN
989,14589,562,1241,0,86,16,NaN,NaN,43.242386,-77.169969,0.0,1.0,0.0,100.0,200.0,NaN,NaN
990,14590,112,657,0,46,0,NaN,NaN,43.233995,-76.821190,0.0,1.0,0.0,100.0,200.0,NaN,NaN
991,14608,726,1951,0,480,284,NaN,NaN,43.151728,-77.626069,0.0,0.0,1.0,200.0,400.0,NaN,NaN


In [88]:
df_results["ratio_total"] = (df_results["current_0-5"] + df_results["current_5-12"]) / df_results["population_total"]
df_results["ratio_0-5"] = df_results["current_0-5"] / df_results["population_0-5"]

df_results["updated_ratio_total"] = df_results["updated_total"] / df_results["population_total"]
df_results["updated_ratio_0-5"] = df_results["updated_0-5"] / df_results["population_0-5"]
df_results

,zip_code,population_0-5,population_total,high demand,current_0-5,current_5-12,expand_0-5,expand_5-12,latitude,longitude,...,medium,large,new_0-5,new_total,updated_0-5,updated_total,ratio_total,ratio_0-5,updated_ratio_total,updated_ratio_0-5
0,10001,744,2140,0,9,600,0.0,0.0,40.748323,-73.997539,...,1.0,2.0,500.0,1000.0,509.0,1609.0,0.284579,0.012097,0.751869,0.684140
1,10002,2142,7267,0,93,4628,0.0,0.0,40.715579,-73.987763,...,0.0,7.0,1400.0,2800.0,1493.0,7521.0,0.649649,0.043417,1.034953,0.697012
2,10003,1440,2970,0,0,1995,0.0,0.0,40.730699,-73.988734,...,0.0,5.0,1000.0,2000.0,1000.0,3995.0,0.671717,0.000000,1.345118,0.694444
3,10004,433,672,0,0,263,0.0,0.0,40.715242,-74.005619,...,1.0,1.0,300.0,600.0,300.0,863.0,0.391369,0.000000,1.284226,0.692841
4,10005,484,794,1,0,39,0.0,0.0,40.705546,-74.007167,...,0.0,2.0,400.0,800.0,400.0,839.0,0.049118,0.000000,1.056675,0.826446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,14586,386,1934,0,11,29,NaN,NaN,43.040930,-77.687060,...,1.0,0.0,100.0,200.0,NaN,NaN,0.020683,0.028497,NaN,NaN
989,14589,562,1241,0,86,16,NaN,NaN,43.242386,-77.169969,...,1.0,0.0,100.0,200.0,NaN,NaN,0.082192,0.153025,NaN,NaN
990,14590,112,657,0,46,0,NaN,NaN,43.233995,-76.821190,...,1.0,0.0,100.0,200.0,NaN,NaN,0.070015,0.410714,NaN,NaN
991,14608,726,1951,0,480,284,NaN,NaN,43.151728,-77.626069,...,0.0,1.0,200.0,400.0,NaN,NaN,0.391594,0.661157,NaN,NaN


In [89]:
df_results.max()

zip_code               14609.00000
population_0-5         12125.00000
population_total       26962.00000
high demand                1.00000
current_0-5             1617.00000
current_5-12            6244.00000
expand_0-5                 4.00000
expand_5-12                5.00000
latitude                  44.98837
longitude                -71.94420
small                      1.00000
medium                     1.00000
large                     38.00000
new_0-5                 7700.00000
new_total              15400.00000
updated_0-5             7915.00000
updated_total          18487.00000
ratio_total                    inf
ratio_0-5                      inf
updated_ratio_total            inf
updated_ratio_0-5              inf
dtype: float64

In [90]:
df_results.query("`new_0-5` == 0")

,zip_code,population_0-5,population_total,high demand,current_0-5,current_5-12,expand_0-5,expand_5-12,latitude,longitude,...,medium,large,new_0-5,new_total,updated_0-5,updated_total,ratio_total,ratio_0-5,updated_ratio_total,updated_ratio_0-5
81,10504,521,1490,0,547,159,0.0,0.0,41.136907,-73.699772,...,0.0,0.0,0.0,0.0,547.0,706.0,0.473826,1.049904,0.473826,1.049904
85,10510,375,1458,0,378,326,0.0,0.0,41.144767,-73.834430,...,0.0,0.0,0.0,0.0,378.0,704.0,0.482853,1.008000,0.482853,1.008000
90,10520,618,1895,0,546,241,0.0,0.0,41.217498,-73.891619,...,0.0,0.0,0.0,0.0,546.0,787.0,0.415303,0.883495,0.415303,0.883495
102,10543,1142,3629,0,430,437,0.0,0.0,40.952594,-73.734962,...,0.0,0.0,0.0,0.0,430.0,867.0,0.238909,0.376532,0.238909,0.376532
115,10576,98,491,0,0,40,0.0,0.0,41.205137,-73.573486,...,0.0,0.0,0.0,0.0,0.0,40.0,0.081466,0.000000,0.081466,0.000000
124,10594,359,937,0,10,22,0.0,0.0,41.119122,-73.773310,...,0.0,0.0,0.0,0.0,10.0,32.0,0.034152,0.027855,0.034152,0.027855
129,10604,707,2375,0,441,286,0.0,0.0,41.060355,-73.739919,...,0.0,0.0,0.0,0.0,441.0,727.0,0.306105,0.623762,0.306105,0.623762
136,10705,1959,6024,0,447,1247,0.0,5.0,40.917682,-73.894579,...,0.0,0.0,0.0,0.0,447.0,1699.0,0.281208,0.228178,0.282039,0.228178
143,10803,656,2829,0,559,101,0.0,0.0,40.903877,-73.807995,...,0.0,0.0,0.0,0.0,559.0,660.0,0.233298,0.852134,0.233298,0.852134
146,10901,1279,4145,0,334,304,0.0,0.0,41.115965,-74.124589,...,0.0,0.0,0.0,0.0,334.0,638.0,0.153920,0.261142,0.153920,0.261142


In [91]:
zip_codes_nyc = df_care.query("city == 'New York'")['zip_code'].drop_duplicates().to_list()